In [15]:
%pip install rich

import requests
from rich import print

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    print(data['current'])
    return data['current']['temperature_2m']

/Users/Silva/Downloads/temporary/function-calling/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [16]:
# Example: Get weather for New York City (latitude: 40.7128, longitude: -74.0060)
get_weather(40.7128, -74.0060)

{'time': '2025-06-15T12:30', 'interval': 900, 'temperature_2m': 15.7, 'wind_speed_10m': 8.1}

15.7

In [17]:
from openai import OpenAI
import json
import os
from rich import print
from dotenv import load_dotenv

load_dotenv()

token = os.getenv("MY_TOKEN")
endpoint = "https://models.github.ai/inference"
model = "openai/gpt-4.1-nano"

client = OpenAI(
    base_url=endpoint,
    api_key=token,
)

tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for provided coordinates in celsius.",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {"type": "number"},
                "longitude": {"type": "number"}
            },
            "required": ["latitude", "longitude"],
            "additionalProperties": False
        },
        "strict": True
    }
}]

messages = [{"role": "user", "content": "What's the weather like in Paris today?"}]

completion = client.chat.completions.create(
    model=model,
    messages=messages, # type: ignore
    tools=tools, # type: ignore
)

print(completion)

ChatCompletion(
    id='chatcmpl-BigqVplro7bjnEsx7OEk2IU0BlHZk',
    choices=[
        Choice(
            finish_reason='tool_calls',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content=None,
                refusal=None,
                role='assistant',
                annotations=[],
                audio=None,
                function_call=None,
                tool_calls=[
                    ChatCompletionMessageToolCall(
                        id='call_bC4tnkxVhyXFnyETRBOFKt3s',
                        function=Function(
                            arguments='{"latitude": 48.8566, "longitude": 2.3522}',
                            name='get_weather'
                        ),
                        type='function'
                    )
                ]
            ),
            content_filter_results={}
        )
    ],
    created=1749990831,
    model='gpt-4.1-nano-2025-04-14',
    object='chat.completion',
    service_tier=None,
    system_fingerprint='fp_68472df8fd',
    usage=CompletionUsage(
        completion_tokens=41,
        prompt_tokens=59,
        total_tokens=100,
        completion_tokens_details=CompletionTokensDetails(
            accepted_prediction_tokens=0,
            audio_tokens=0,
            reasoning_tokens=0,
            rejected_prediction_tokens=0
        ),
        prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)
    ),
    prompt_filter_results=[
        {
            'prompt_index': 0,
            'content_filter_results': {
                'hate': {'filtered': False, 'severity': 'safe'},
                'jailbreak': {'filtered': False, 'detected': False},
                'self_harm': {'filtered': False, 'severity': 'safe'},
                'sexual': {'filtered': False, 'severity': 'safe'},
                'violence': {'filtered': False, 'severity': 'safe'}
            }
        }
    ]
)

In [18]:
[{
    "id": "call_12345xyz",
    "type": "function",
    "function": {
      "name": "get_weather",
      "arguments": "{\"latitude\":48.8566,\"longitude\":2.3522}"
    }
}]

[{'id': 'call_12345xyz',
  'type': 'function',
  'function': {'name': 'get_weather',
   'arguments': '{"latitude":48.8566,"longitude":2.3522}'}}]

In [20]:
tool_call = completion.choices[0].message.tool_calls[0]
args = json.loads(tool_call.function.arguments)

result = get_weather(args["latitude"], args["longitude"])
print(f"[bold bluse]Current temperature: {result}°C[bold blue]")

{'time': '2025-06-15T12:30', 'interval': 900, 'temperature_2m': 21.8, 'wind_speed_10m': 7.6}

Current temperature: 21.8°C

In [21]:
messages.append(completion.choices[0].message)  # type: ignore # append model's function call message
messages.append({                               # append result message
    "role": "tool",
    "tool_call_id": tool_call.id,
    "content": str(result)
})

completion_2 = client.chat.completions.create(
    model=model,
    messages=messages, # type: ignore
    tools=tools, # type: ignore
)

In [22]:
completion_2.choices[0].message.content

'The current temperature in Paris is approximately 21.8°C.'